# Downloading the Dataset

[Dataset Link](https://www.kaggle.com/datasets/shashwatwork/identifying-disease-in-tea-leafs)

In [5]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
! mkdir ~/.kaggle

In [7]:
cp kaggle.json ~/.kaggle/

In [8]:
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
! kaggle datasets download -d shashwatwork/identifying-disease-in-tea-leafs

 99% 731M/740M [00:06<00:00, 176MB/s]
100% 740M/740M [00:07<00:00, 111MB/s]


In [10]:
!unzip \*.zip && rm *.zip

Archive:  identifying-disease-in-tea-leafs.zip
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143242.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143328.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143336.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143344.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143352.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143401.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143424.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143433.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143441.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143451.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143501.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143511.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143525.jpg  
  inflating: tea 

# Splitting the Dataset

In [11]:
pip install split-folders[full]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import splitfolders

In [13]:
input_folder = '/content/tea sickness dataset'
splitfolders.ratio(input_folder, output="Dataset", 
                   seed=42, ratio=(.8, 0,.2), 
                   group_prefix=None)

Copying files: 885 files [00:02, 336.95 files/s]


In [ ]:
import os
FOLDER_PATH = 'Anthracnose'
ROOT_PATH = '/content/Dataset/val/'
print(len(os.listdir(os.path.join(ROOT_PATH, FOLDER_PATH))))

0


# Model Building

**Importing Libraries**

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
import numpy as np
import matplotlib.pyplot as plt

**Defining Train & Test Set**

In [ ]:
imageSize = [224, 224]

trainPath = r"/content/Dataset/train"

testPath = r"/content/Dataset/test"

**Defining VGG **

In [ ]:
# adding preprocessing layers to the front of vgg

vgg = VGG16(input_shape=imageSize + [3], weights='imagenet',include_top=False)

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

**Adding Layers**

In [ ]:
# our layers - you can add more if you want
#Adding Flatten Layers
x = Flatten()(vgg.output)

In [ ]:
#Adding Output Layer
prediction = Dense(8, activation='softmax')(x)

**Defining the Model**

In [ ]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'], run_eagerly=True
)


**Image Augmentation**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(trainPath,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 706 images belonging to 8 classes.
Found 179 images belonging to 8 classes.


In [ ]:
training_set.class_indices

{'Anthracnose': 0,
 'algal leaf': 1,
 'bird eye spot': 2,
 'brown blight': 3,
 'gray light': 4,
 'healthy': 5,
 'red leaf spot': 6,
 'white spot': 7}

**Model Training**

In [ ]:
import sys
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=150,
  steps_per_epoch=len(training_set)//3,
  validation_steps=len(test_set)//3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/150
7/7 [==============================] - 31s 4s/step - loss: 2.9193 - accuracy: 0.1786 - val_loss: 2.4473 - val_accuracy: 0.1719
Epoch 2/150
7/7 [==============================] - 28s 4s/step - loss: 1.8477 - accuracy: 0.3661 - val_loss: 1.4763 - val_accuracy: 0.4219
Epoch 3/150
7/7 [==============================] - 28s 4s/step - loss: 1.1795 - accuracy: 0.5536 - val_loss: 1.6247 - val_accuracy: 0.3750
Epoch 4/150
7/7 [==============================] - 29s 4s/step - loss: 1.0125 - accuracy: 0.6384 - val_loss: 1.1357 - val_accuracy: 0.5469
Epoch 5/150
7/7 [==============================] - 29s 4s/step - loss: 0.9598 - accuracy: 0.6652 - val_loss: 0.9185 - val_accuracy: 0.6719
Epoch 6/150
7/7 [==============================] - 28s 4s/step - loss: 0.8386 - accuracy: 0.6920 - val_loss: 0.8879 - val_accuracy: 0.6875
Epoch 7/150
7/7 [==============================] - 28s 4s/step - loss: 0.7176 - accuracy: 0.7589 - val_loss: 1.1682 - val_accuracy: 0.5938
Epoch 8/150
7/7 [==========

**Save the Model**

In [ ]:
#save the model
model.save('vgg-16-Tea-leaves-disease-model.h5')

**Load the Model**

In [40]:
#import load_model class for loading h5 file
from tensorflow.keras.models import load_model
#import image class to process the images
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np

In [41]:
#load saved model file
model=load_model('vgg-16-Tea-leaves-disease-model.h5')

# Testing the Model

**Test Case 1**

In [15]:
img=image.load_img(r'/content/Dataset/test/red leaf spot/UNADJUSTEDNONRAW_thumb_1b5.jpg',target_size=(224,224))
#convert image to array format
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
output=np.argmax(model.predict(img_data), axis=1)
index=['Anthracnose',
 'algal leaf',
 'bird eye spot',
 'brown blight',
 'gray light',
 'healthy',
 'red leaf spot',
 'white spot']
result = str(index[output[0]])
result

1/1 [==============================] - 0s 19ms/step


'red leaf spot'

**Test Case 2**

In [16]:
img=image.load_img(r'/content/Dataset/test/Anthracnose/IMG_20220503_145528.jpg',target_size=(224,224))
#convert image to array format
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
output=np.argmax(model.predict(img_data), axis=1)
index=['Anthracnose',
 'algal leaf',
 'bird eye spot',
 'brown blight',
 'gray light',
 'healthy',
 'red leaf spot',
 'white spot']
result = str(index[output[0]])
result

1/1 [==============================] - 0s 24ms/step


'Anthracnose'

**Test Case 3**

In [17]:
img=image.load_img(r'/content/Dataset/test/algal leaf/UNADJUSTEDNONRAW_thumb_50.jpg',target_size=(224,224))
#convert image to array format
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
output=np.argmax(model.predict(img_data), axis=1)
index=['Anthracnose',
 'algal leaf',
 'bird eye spot',
 'brown blight',
 'gray light',
 'healthy',
 'red leaf spot',
 'white spot']
result = str(index[output[0]])
result

1/1 [==============================] - 0s 16ms/step


'algal leaf'

**Test Case 4**

In [19]:
img=image.load_img(r'/content/Dataset/test/bird eye spot/IMG_20220503_154332.jpg',target_size=(224,224))
#convert image to array format
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
output=np.argmax(model.predict(img_data), axis=1)
index=['Anthracnose',
 'algal leaf',
 'bird eye spot',
 'brown blight',
 'gray light',
 'healthy',
 'red leaf spot',
 'white spot']
result = str(index[output[0]])
result

1/1 [==============================] - 0s 16ms/step


'bird eye spot'

**Test Case 5**

In [25]:
img=image.load_img(r'/content/Dataset/test/brown blight/UNADJUSTEDNONRAW_thumb_113.jpg',target_size=(224,224))
#convert image to array format
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
output=np.argmax(model.predict(img_data), axis=1)
index=['Anthracnose',
 'algal leaf',
 'bird eye spot',
 'brown blight',
 'gray light',
 'healthy',
 'red leaf spot',
 'white spot']
result = str(index[output[0]])
result

1/1 [==============================] - 0s 17ms/step


'brown blight'

**Test Case 6**

In [26]:
img=image.load_img(r'/content/Dataset/test/gray light/IMG_20220503_135357.jpg',target_size=(224,224))
#convert image to array format
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
output=np.argmax(model.predict(img_data), axis=1)
index=['Anthracnose',
 'algal leaf',
 'bird eye spot',
 'brown blight',
 'gray light',
 'healthy',
 'red leaf spot',
 'white spot']
result = str(index[output[0]])
result

1/1 [==============================] - 0s 17ms/step


'gray light'

**Test Case 7**

In [27]:
img=image.load_img(r'/content/Dataset/test/healthy/UNADJUSTEDNONRAW_thumb_209.jpg',target_size=(224,224))
#convert image to array format
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
output=np.argmax(model.predict(img_data), axis=1)
index=['Anthracnose',
 'algal leaf',
 'bird eye spot',
 'brown blight',
 'gray light',
 'healthy',
 'red leaf spot',
 'white spot']
result = str(index[output[0]])
result

1/1 [==============================] - 0s 19ms/step


'healthy'

**Test Case 8**

In [44]:
img=image.load_img(r'/content/Dataset/test/white spot/UNADJUSTEDNONRAW_thumb_de.jpg',target_size=(224,224))
#convert image to array format
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
output=np.argmax(model.predict(img_data), axis=1)
index=['Anthracnose',
 'algal leaf',
 'bird eye spot',
 'brown blight',
 'gray light',
 'healthy',
 'red leaf spot',
 'white spot']
result = str(index[output[0]])
result

1/1 [==============================] - 0s 16ms/step


'white spot'